In [170]:
import json
import enchant
# import requests module
import requests
# Import pandas
import pandas as pd

In [171]:
ENGLISH_DICT = enchant.Dict("en_US")
TRIVIAL_WORDS = ["company", "inc", "inc.", "group", "corporation", "co", "|", "corp", "and" , "&", "llc", "llc.","ltd","ltd.", "the", "of", "a", "an"]

In [172]:
def arrangeWordsByImportance(company):
    #lst = sorted(company.lower().split(), key=lambda x: len(x), reverse=True)
    company = company.replace('|', ' ' )
    lst = company.lower().split()
    #print(lst)
    nonwords = []
    others = []
    for word in lst:
        # Removes trivial words
        if word in TRIVIAL_WORDS or word in others or word in nonwords:
            continue
        # Marks words that aren't trivial and aren't in dictionary as more important
        elif not ENGLISH_DICT.check(word):
            nonwords.append(word)
        else:
            others.append(word)
    return (nonwords, others)

In [188]:
def arrangeWordsByOrder(company):
    #lst = sorted(company.lower().split(), key=lambda x: len(x), reverse=True)
    company = company.replace('|', ' ' )
    lst = company.lower().split()
    words = []
    for word in lst:
        # Removes trivial words
        if word in TRIVIAL_WORDS or word in words:
            continue
        # Marks words that aren't trivial and aren't in dictionary as more important
        else:
            words.append(word)
    return words

In [173]:
def makeCompanyName(nameWords,iteration):
    #nameWords = arrangeWordsByImportance(rawName)
    #print(nameWords)
    name = ''
    if nameWords[0] != '':
        for item in nameWords[0]:
            name = name+''+item
    for i in range(len(nameWords[1])-iteration):
        if name == '':
            name = nameWords[1][i]
        else:
            name = name+' '+nameWords[1][i]
    return(name)

In [183]:
def makeCompanyNameFromWords(nameWords,iteration):
    #nameWords = arrangeWordsByImportance(rawName)
    #print(nameWords)
    name = ''
    for i in range(len(nameWords)-iteration):
        if name == '':
            name = nameWords[i]
        else:
            name = name+' '+nameWords[i]
    return(name)

In [174]:
def getURL(queryName):
    r = requests.get('https://autocomplete.clearbit.com/v1/companies/suggest?query=' + queryName)
    json = r.json()
    return json

In [175]:
# reading csv file
#pd.read_csv("example1.csv")

df = pd.read_csv('URL2_CompanyName_UTF8.csv')

In [234]:
pd_sample = df['recipient_legal_name'].sample(n=5)
pd_sample

14631    Placentia Area Chamber of Commerce|Placentia A...
20712    Humber Economic Development Board Inc.|Humber ...
6391     Gander and Area Chamber of Commerce Inc.|Gande...
16627            Trade Centre Limited|Trade Centre Limited
22290    Érablière de la Montagne Verte Inc.|Érablière ...
Name: recipient_legal_name, dtype: object

In [236]:
urlList = []
for item in pd_sample:
    #print(pd_sample.index(item))
    nameWords = arrangeWordsByOrder(item)
    max_iteration = len(nameWords[1])
    i = 0
    while (i <= max_iteration):
        enhancedName = makeCompanyNameFromWords(nameWords,i)
        res = getURL(enhancedName)
        if res != []:
            print(item , enhancedName)
            print(nameWords)
            print(res)
            for row in res:
                urlList.append([item, enhancedName, row['name'], row['domain'], row['logo']])
            print('----------------------------------------------')
            #pd_sample
            break    
        i = i + 1

Placentia Area Chamber of Commerce|Placentia Area Chamber of Commerce placentia
['placentia', 'area', 'chamber', 'commerce']
[{'name': 'Placentia-Yorba Linda Unified School District', 'domain': 'pylusd.org', 'logo': 'https://logo.clearbit.com/pylusd.org'}, {'name': 'Placentia Library', 'domain': 'placentialibrary.org', 'logo': 'https://logo.clearbit.com/placentialibrary.org'}, {'name': 'Placentia Movers', 'domain': 'placentiamoversmoving.com', 'logo': 'https://logo.clearbit.com/placentiamoversmoving.com'}, {'name': 'Placentia Chiropractic Center', 'domain': 'bestocchiropractor.com', 'logo': 'https://logo.clearbit.com/bestocchiropractor.com'}, {'name': 'Placentia PONY Baseball,', 'domain': 'placentiapony.com', 'logo': 'https://logo.clearbit.com/placentiapony.com'}]
----------------------------------------------
Humber Economic Development Board Inc.|Humber Economic Development Board Inc. humber
['humber', 'economic', 'development', 'board']
[{'name': 'Humber College', 'domain': 'humber.

In [238]:
urlList

[['Placentia Area Chamber of Commerce|Placentia Area Chamber of Commerce',
  'placentia',
  'Placentia-Yorba Linda Unified School District',
  'pylusd.org',
  'https://logo.clearbit.com/pylusd.org'],
 ['Placentia Area Chamber of Commerce|Placentia Area Chamber of Commerce',
  'placentia',
  'Placentia Library',
  'placentialibrary.org',
  'https://logo.clearbit.com/placentialibrary.org'],
 ['Placentia Area Chamber of Commerce|Placentia Area Chamber of Commerce',
  'placentia',
  'Placentia Movers',
  'placentiamoversmoving.com',
  'https://logo.clearbit.com/placentiamoversmoving.com'],
 ['Placentia Area Chamber of Commerce|Placentia Area Chamber of Commerce',
  'placentia',
  'Placentia Chiropractic Center',
  'bestocchiropractor.com',
  'https://logo.clearbit.com/bestocchiropractor.com'],
 ['Placentia Area Chamber of Commerce|Placentia Area Chamber of Commerce',
  'placentia',
  'Placentia PONY Baseball,',
  'placentiapony.com',
  'https://logo.clearbit.com/placentiapony.com'],
 ['Hum

In [239]:
urlListdf = pd. DataFrame(urlList, columns=['Original Name', 'Search Name', 'url Company Name', 'url', 'url Logo']) 
urlListdf

,Original Name,Search Name,url Company Name,url,url Logo
0,Placentia Area Chamber of Commerce|Placentia A...,placentia,Placentia-Yorba Linda Unified School District,pylusd.org,https://logo.clearbit.com/pylusd.org
1,Placentia Area Chamber of Commerce|Placentia A...,placentia,Placentia Library,placentialibrary.org,https://logo.clearbit.com/placentialibrary.org
2,Placentia Area Chamber of Commerce|Placentia A...,placentia,Placentia Movers,placentiamoversmoving.com,https://logo.clearbit.com/placentiamoversmovin...
3,Placentia Area Chamber of Commerce|Placentia A...,placentia,Placentia Chiropractic Center,bestocchiropractor.com,https://logo.clearbit.com/bestocchiropractor.com
4,Placentia Area Chamber of Commerce|Placentia A...,placentia,"Placentia PONY Baseball,",placentiapony.com,https://logo.clearbit.com/placentiapony.com
5,Humber Economic Development Board Inc.|Humber ...,humber,Humber College,humber.ca,https://logo.clearbit.com/humber.ca
6,Humber Economic Development Board Inc.|Humber ...,humber,Humber River Hospital,hrh.ca,https://logo.clearbit.com/hrh.ca
7,Humber Economic Development Board Inc.|Humber ...,humber,Humberside Police,humberside.police.uk,https://logo.clearbit.com/humberside.police.uk
8,Humber Economic Development Board Inc.|Humber ...,humber,Humber Cinemas,humbercinemas.com,https://logo.clearbit.com/humbercinemas.com
9,Humber Economic Development Board Inc.|Humber ...,humber,HUMBER,humber.com.ar,https://logo.clearbit.com/humber.com.ar


In [240]:
urlListdf.to_csv('url_list_sample.csv')

In [245]:
url = 'https://httpbin.org/ip'
response = requests.get(url)#,proxies='http://185.191.76.84:80')
print(response.json())

{'origin': '163.120.100.252'}
